## Import Needed Libraries

In [ ]:
from  texas_gerrymandering_hb4.config import FINAL_CSV
import json
import os
import joblib

from sklearn.preprocessing import StandardScaler

In [ ]:
RACE_COLS = ["pct_white", "pct_black", "pct_asian", "pct_hispanic"]
ID_COL = "district_id"

In [ ]:
def build_features(df: pd.DataFrame, dop_race: bool) -> pd.DataFrame:
    cols = [c for c in df.columns if c!= ID_COL]
    if drop_race:
        cols = [c for c in cols if c not in RACE_COLS]
    return df[cols].copy(), cols

In [ ]:
def main():
    df = pd.read_csv(FINAL_CSV)
    X_full, full_cols = build_features(df, drop_race=False)
    scaler_full = StandardScaler()
    X_full_scaled = scaler_full.fit_transform(X_full.values)
    joblib.dump(scaler_full, os.path)